In [30]:
import pandas as pd
import requests
import json

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [14]:
key = '05cf3b9c-f337-4b65-ae21-e1721a9f6967'

In [5]:
results_table = pd.read_csv('../../../Data/Humach/wslive_with_results_2019-06-07_to_2020-02-21.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,10,16,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
residential = f'https://api.accurateappend.com/Services/V2/AppendPhone/Residential/{key}'
business = 'https://api.accurateappend.com/Services/V2/AppendPhone/Business/'

In [8]:
first = 'larry'
last= 'grose'
address= '10819 s drake'
city= 'chicago'
state= 'il'
zipcode= '60655'
parameters = {'firstname':first, 'lastname': last,'address':address, 'city':city,'state':state,'postalcode':zipcode}
response =  requests.get(residential, params=parameters)
results = response.json()

In [24]:
results['Phones']

[{'AreaCode': '773',
  'LineType': 'LandLine',
  'MatchLevel': 'E1',
  'MaxValidationLevel': 'E1',
  'PhoneNumber': '2395590',
  'Source': 'D'},
 {'AreaCode': '773',
  'LineType': 'CellLine',
  'MatchLevel': 'E1',
  'MaxValidationLevel': 'E1',
  'PhoneNumber': '7185598',
  'Source': 'M'}]

In [16]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
"HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
"MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [23]:
RECENT = results_table[results_table.SOURCE_FILE_DT=='2020-01-27']
phys_sample = RECENT[(RECENT.phone_status == 'Confirmed')&(RECENT.addr_status == 'Confirmed')&(RECENT.OFFICE_ADDRESS_LINE_1)&(RECENT.OFFICE_ADDRESS_STATE.isin(states))].sample(10)

In [46]:
phys = RECENT[(RECENT.phone_status == 'Confirmed')&(RECENT.addr_status == 'Confirmed')&(RECENT.OFFICE_ADDRESS_LINE_1)&(RECENT.OFFICE_ADDRESS_STATE.isin(states))]

In [50]:
new_sample = phys[phys['PHYSICIAN_ME_NUMBER'].isin(list(phys_sample.PHYSICIAN_ME_NUMBER))==False].sample(100)

In [55]:
for row in new_sample.itertuples():
    first = row.PHYSICIAN_FIRST_NAME.lower()
    last = row.PHYSICIAN_LAST_NAME.lower()
    address = row.OFFICE_ADDRESS_LINE_2.lower()
    city = row.OFFICE_ADDRESS_CITY.lower()
    state = row.OFFICE_ADDRESS_STATE.lower()
    zipcode = row.OFFICE_ADDRESS_ZIP.lower()
    parameters = {'firstname':first, 'lastname': last,'address':address, 'city':city,'state':state,'postalcode':zipcode}
    response =  requests.get(residential, params=parameters)
    results = response.json()
    all_results.append(results)
    for phone in results['Phones']:
        print(f'{row.PHYSICIAN_FIRST_NAME.title()} {row.PHYSICIAN_LAST_NAME.title()} found!')
        new_dict = {
            'ME':row.PHYSICIAN_ME_NUMBER,
            'AreaCode': phone['AreaCode'],
            'LineType': phone['LineType'],
            'MatchLevel': phone['MatchLevel'],
            'MaxValidationLevel': phone['MaxValidationLevel'],
            'PhoneNumber': phone['PhoneNumber'],
            'Source': phone['Source'],
            'TELEPHONE': str(phone['AreaCode'])+str(phone['PhoneNumber'])
        }
        new_dict_list.append(new_dict)

Mary Vader found!
Mary Vader found!
Michael Hord found!
Roger Lubbers found!
Roger Lubbers found!
Christopher Quinn found!
Alfredo Negrete found!
Kwadwo Agyei-Gyamfi found!
Phillip Greig found!
Michael Lee found!
Jean Thur found!
Jude Sauer found!
Joel Carney found!
Joel Carney found!
Nicole O'Brien found!
Priti Patel found!
David Rowe found!
David Rowe found!
Carmen Allegra found!
Carmen Allegra found!
Peter Gottlieb found!
Peter Gottlieb found!
Laura Devilbiss found!
Frank Knoll found!
Charles Romberger found!
Rick Thyes found!
Rick Thyes found!
James Watkins found!
Katherine Hajjar found!
Katherine Hajjar found!
Katherine Hajjar found!
David Boes found!
Debra Szeluga found!
Luis Concepcion found!
Luis Concepcion found!
Mark Elias found!
Kasiani Myers found!
Alan Altman found!
Deborah Agnew found!
Ileana Zapatero found!
Ileana Zapatero found!
Jeffrey Schleusener found!
Jeffrey Schleusener found!
Andrew Hart found!
Eiman Shafa found!
Joseph Palumbo found!
Malcolm Robinson found!
Luke 

In [36]:
B = phys_sample[['PHYSICIAN_ME_NUMBER','OFFICE_TELEPHONE']]

In [58]:
boo = new_sample[['PHYSICIAN_ME_NUMBER','OFFICE_TELEPHONE']]
boo.OFFICE_TELEPHONE = boo.OFFICE_TELEPHONE.astype('int64')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [38]:
D = pd.merge(B,C, left_on='PHYSICIAN_ME_NUMBER', right_on='ME')

In [42]:
D.OFFICE_TELEPHONE = D.OFFICE_TELEPHONE.astype('int64')

In [45]:
D.dtypes

PHYSICIAN_ME_NUMBER    float64
OFFICE_TELEPHONE         int64
ME                     float64
AreaCode                object
LineType                object
MatchLevel              object
MaxValidationLevel      object
PhoneNumber             object
Source                  object
dtype: object

In [57]:
E = pd.DataFrame(new_dict_list)

In [62]:
THIS= pd.merge(boo, E, left_on = 'PHYSICIAN_ME_NUMBER', right_on='ME')
THIS[THIS.MatchLevel=='E1']

,PHYSICIAN_ME_NUMBER,OFFICE_TELEPHONE,ME,AreaCode,LineType,MatchLevel,MaxValidationLevel,PhoneNumber,Source,TELEPHONE
0,4.878870e+09,9702492421,4.878870e+09,970,LandLine,E1,E1,2492421,D,9702492421
1,4.878870e+09,9702492421,4.878870e+09,970,CellLine,E1,E1,5904243,M,9705904243
9,6.001820e+09,5155746080,6.001820e+09,515,LandLine,E1,E1,5746820,D,5155746820
11,3.545851e+09,5858696600,3.545851e+09,585,LandLine,E1,E1,8696600,P,5858696600
25,5.605961e+09,9204574461,5.605961e+09,920,LandLine,E1,E1,4574461,D,9204574461
35,2.405890e+09,6033323355,2.405890e+09,603,LandLine,E1,E1,3323355,P,6033323355
39,1.102841e+09,3104545555,1.102841e+09,310,LandLine,E1,E1,4545555,P,3104545555
46,2.401871e+09,6177328272,2.401871e+09,617,LandLine,E1,E1,7328272,P,6177328272
52,3.506891e+09,7168820366,3.506891e+09,716,LandLine,E1,E1,8820366,P,7168820366
64,7.481172e+10,7315842117,7.481172e+10,731,LandLine,E1,E1,5842117,P,7315842117


In [67]:
for row in THIS.itertuples():
    if str(row.OFFICE_TELEPHONE) == str(row.TELEPHONE):
        print(row.MatchLevel)

E1
B2
E1
E1
E1
E1
E1
E1
N1
E1


In [69]:
THIS.drop_duplicates('PHYSICIAN_ME_NUMBER')

,PHYSICIAN_ME_NUMBER,OFFICE_TELEPHONE,ME,AreaCode,LineType,MatchLevel,MaxValidationLevel,PhoneNumber,Source,TELEPHONE
0,4.878870e+09,9702492421,4.878870e+09,970,LandLine,E1,E1,2492421,D,9702492421
2,4.813982e+09,8066370344,4.813982e+09,806,LandLine,B2,B2,7853825,P,8067853825
3,1.606721e+09,3092773500,1.606721e+09,309,LandLine,B2,B2,2773500,D,3092773500
5,1.720892e+09,5742710700,1.720892e+09,248,CellLine,B2,B2,7909210,M,2487909210
6,1.611981e+09,7603527216,1.611981e+09,760,CellLine,B2,B2,9960005,M,7609960005
7,4.120193e+10,9107395197,4.120193e+10,910,LandLine,B2,B2,7395389,P,9107395389
8,4.812871e+09,8643313230,4.812871e+09,864,LandLine,B2,B2,2342984,D,8642342984
9,6.001820e+09,5155746080,6.001820e+09,515,LandLine,E1,E1,5746820,D,5155746820
10,1.676921e+09,9203038700,1.676921e+09,920,LandLine,B2,B2,3030558,D,9203030558
11,3.545851e+09,5858696600,3.545851e+09,585,LandLine,E1,E1,8696600,P,5858696600


In [75]:
me_list = list(E.drop_duplicates('ME')['ME'])

In [78]:
all_ = pd.concat([new_sample, phys_sample])

In [88]:
found = all_[all_['PHYSICIAN_ME_NUMBER'].isin(me_list)]

In [89]:
not_found = all_[all_['PHYSICIAN_ME_NUMBER'].isin(me_list)==False].sample(49)

In [93]:
pd.concat([found, not_found]).to_csv('../../../Data/Humach/confirmed_phone_sample.csv')

In [83]:
all_

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
309262,4.878870e+09,MARY,LOUISE,VADER,NaN,NaN,THE PEDATRIC ASSOC,947 S 5TH ST,MONTROSE,CO,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
306003,4.813982e+09,MICHAEL,S,HORD,NaN,NaN,BROWNFIELD FAMILY PHYSICIANS,706 E FELT ST,BROWNFIELD,TX,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
301395,2.401931e+09,CHARLES,MC CLASKEY,FRIEL,NaN,NaN,UBA MEDICAL CENTER,BOX 800709,CHARLOTTESVLE,VA,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
309039,5.611687e+10,MICHAEL,A,LAMACCHIA,NaN,NaN,ST JOSEPH MED REG CTR,703 MAIN ST,PATERSON,NJ,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
304639,4.504880e+09,GARY,ALLEN,BASS,NaN,NaN,CHILDREN'S MEDICAL CENTER,703 VERDAE BLVD,GREENVILLE,SC,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304290,2.405911e+09,EDWARD,ARAM,HATCHIGIAN,NaN,NaN,MEDICAL CENTER,330 BROOKLINE AVE,BOSTON,MA,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
301452,5.404911e+09,DONALD,LOREN,SLACK,NaN,NaN,MT VERNON FIRE DEPT,1901 N LAVENTURE RD,MOUNT VERNON,WA,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
309509,3.607570e+09,R,RODNEY,HOWELL,NaN,NaN,BRB STE 626,1501 NW 10TH AVE,MIAMI,FL,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
304106,4.113851e+09,DEBORAH,MITCHELL,BURTON,NaN,NaN,ATLANTA KIDS ENT,2550 WINDY HILL RD SE STE 307,MARIETTA,GA,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed


In [85]:
E.drop_duplicates('ME')

,ME,AreaCode,LineType,MatchLevel,MaxValidationLevel,PhoneNumber,Source,TELEPHONE
0,5.606910e+09,626,LandLine,E1,E1,2851154,D,6262851154
2,5.606750e+09,414,LandLine,E1,E1,9557250,D,4149557250
5,6.491408e+10,520,CellLine,B2,B2,3430610,M,5203430610
6,4.113851e+09,770,LandLine,B2,B2,4280936,P,7704280936
8,4.878870e+09,970,LandLine,E1,E1,2492421,D,9702492421
10,4.813982e+09,806,LandLine,B2,B2,7853825,P,8067853825
11,1.606721e+09,309,LandLine,B2,B2,2773500,D,3092773500
13,1.720892e+09,248,CellLine,B2,B2,7909210,M,2487909210
14,1.611981e+09,760,CellLine,B2,B2,9960005,M,7609960005
15,4.120193e+10,910,LandLine,B2,B2,7395389,P,9107395389


In [4]:
PPD = pd.read_csv('../../../Data/PPD/ppd_data_20200229.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
test = PPD[(PPD['TELEPHONE_NUMBER'].isna())&(PPD['POLO_MAILING_LINE_1'].isna()==False)&(PPD['TOP_CD']==20)&(PPD.POLO_STATE.isin(states))].sample(100)

In [20]:
test = test.fillna('None')

In [22]:
all_results =[]
new_dict_list = []
for row in test.itertuples():
    first = row.FIRST_NAME.lower()
    last = row.LAST_NAME.lower()
    address = row.POLO_MAILING_LINE_2.lower()
    city = row.POLO_CITY.lower()
    state = row.POLO_STATE.lower()
    zipcode = row.POLO_ZIP
    parameters = {'firstname':first, 'lastname': last,'address':address, 'city':city,'state':state,'postalcode':zipcode}
    response =  requests.get(residential, params=parameters)
    results = response.json()
    all_results.append(results)
    for phone in results['Phones']:
        print(f'{row.FIRST_NAME.title()} {row.LAST_NAME.title()} found!')
        new_dict = {
            'ME':row.ME,
            'AreaCode': phone['AreaCode'],
            'LineType': phone['LineType'],
            'MatchLevel': phone['MatchLevel'],
            'MaxValidationLevel': phone['MaxValidationLevel'],
            'PhoneNumber': phone['PhoneNumber'],
            'Source': phone['Source'],
            'TELEPHONE': str(phone['AreaCode'])+str(phone['PhoneNumber'])
        }
        new_dict_list.append(new_dict)

Jennifer Hsu found!
Norman Rosenthal found!
Alexander Nein found!
Alexander Nein found!
Matthew Richard found!
Andrew Brenner found!
Aiko Mcglynn found!
Aiko Mcglynn found!
Eric Scharling found!
John Barr found!
John Barr found!
Kode Ediale found!
Kode Ediale found!
Elizabeth Allen found!
Melissa Lerman found!
Ankit Rohatgi found!
Luis Gonzalez Castro found!
Hong Gao found!
Michael Siemer found!
Michael Siemer found!
Michael Siemer found!
Bruce Graham found!
Bruce Graham found!
Ann Schutt-Aine found!
Cary Siegel found!
Mondira Sengupta found!
Theodore Bushnell found!
Tan Nguyen found!
Tan Nguyen found!
Shreyas Baxi found!
Shreyas Baxi found!
Ann Kimball found!
Jeremy Perkins found!
Sascha Schubert found!
Sascha Schubert found!


In [28]:
# test_df = pd.DataFrame(new_dict_list)
test_df[test_df.Source=='D']

,ME,AreaCode,LineType,MatchLevel,MaxValidationLevel,PhoneNumber,Source,TELEPHONE
2,102870930,615,LandLine,E1,E1,3270201,D,6153270201
6,1175081093,980,LandLine,B2,B2,9496375,D,9809496375
8,3601851350,336,LandLine,E1,E1,3976000,D,3363976000
13,3601850248,336,LandLine,B2,B2,7657278,D,3367657278
18,2834831298,314,LandLine,E1,E1,5696150,D,3145696150
21,6002740107,559,LandLine,B2,B2,7136672,D,5597136672
24,4101851341,718,LandLine,B2,B2,7670010,D,7187670010
27,3546121533,408,LandLine,B2,B2,9842536,D,4089842536


In [29]:
test_df.to_csv('Accurate_Append.csv', index=False)

In [33]:
test.to_csv('../../../Data/Humach/AccurateAppendMissingSample.csv', index=False)

In [34]:
E

NameError: name 'E' is not defined